# Librerias

In [52]:
import os
import time
import json
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Tiempos de ejecución

## Tiempos de Cropeo de celulas de las 400 imagenes

In [ ]:
# Define directories
image_directory = "base"
output_directory = "crops_90x90"

# Ensure output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Load bounding boxes from JSON
with open('classificationDownload/classifications.json', 'r') as file:
    bounding_boxes = json.load(file)

# Function to crop images
def crop_images(image_directory, bounding_boxes):
    total_cropping_time = 0
    for filename in os.listdir(image_directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            with Image.open(os.path.join(image_directory, filename)) as img:
                for entry in bounding_boxes:
                    if entry['image_name'] == filename:
                        start_time = time.time()
                        for idx, classification in enumerate(entry['classifications']):
                            x_center, y_center = classification['nucleus_x'], classification['nucleus_y']
                            x, y = int(x_center - 45), int(y_center - 45)
                            cropped_img = img.crop((x, y, x + 90, y + 90))
                            cropped_img.save(os.path.join(output_directory, f"{filename[:-4]}_{idx}.png"))
                            
                        total_cropping_time += time.time() - start_time
    return total_cropping_time

# Crop images and measure time
cropping_time = crop_images(image_directory, bounding_boxes)
print(f"Total Cropping Time: {cropping_time:.2f} seconds")

Total Cropping Time: 31.28 seconds


## Tiempo de predicción del Binario

In [ ]:
# Define directories and load model
image_directory = "base"
output_directory = "imgs/crops_90x90"
model_path = "modelosFinales/xception_clasificacion2clases.keras"  # Adjust with actual path to your pre-trained model

# Ensure output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Load the model without compiling
model = tf.keras.models.load_model(model_path, compile=False)

# Recompile the model manually with a known optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Function to adjust class name
def adjust_class_name(bethesda_system):
    return 'Negative' if bethesda_system == 'Negative for intraepithelial lesion' else bethesda_system

# Function to process images
def process_images(image_directory, bounding_boxes):
    total_cropping_time = 0
    total_prediction_time = 0
    for filename in os.listdir(image_directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            with Image.open(os.path.join(image_directory, filename)) as img:
                for entry in bounding_boxes:
                    if entry['image_name'] == filename:
                        start_time = time.time()
                        for idx, classification in enumerate(entry['classifications']):
                            x_center, y_center = classification['nucleus_x'], classification['nucleus_y']
                            bethesda_system = adjust_class_name(classification['bethesda_system'])
                            x, y = int(x_center - 45), int(y_center - 45)
                            cropped_img = img.crop((x, y, x + 90, y + 90))
                            # Resize the image to 224x224
                            resized_img = cropped_img.resize((224, 224))
                            cropped_img.save(os.path.join(output_directory, f"{filename[:-4]}_{bethesda_system}_{idx}.png"))
                        total_cropping_time += time.time() - start_time

                        # Predict using the model
                        start_time = time.time()
                        resized_img_array = np.array(resized_img)[np.newaxis, ...]  # Adjust dimensions if needed
                        predictions = model.predict(resized_img_array)
                        total_prediction_time += time.time() - start_time
    return total_prediction_time

# Load bounding boxes from JSON
with open('classificationDownload/classifications.json', 'r') as file:
    bounding_boxes = json.load(file)

# Process images and measure time
prediction_time = process_images(image_directory, bounding_boxes)
print(f"Total Prediction Time: {prediction_time:.2f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 530ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━

## Tiempo de Predicción de la Clasificación por Grado de Lesión

In [ ]:
# Define directories and load model
image_directory = "base"
output_directory = "imgs/crops_90x90"
model_path = "modelosFinales/xception_clasificacion3clases.keras"  

# Ensure output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Load the model without compiling
model = tf.keras.models.load_model(model_path, compile=False)

# Recompile the model manually with a known optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Function to adjust class name
def adjust_class_name(bethesda_system):
    return 'Negative' if bethesda_system == 'Negative for intraepithelial lesion' else bethesda_system

# Function to process images
def process_images(image_directory, bounding_boxes):
    total_cropping_time = 0
    total_prediction_time = 0
    for filename in os.listdir(image_directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            with Image.open(os.path.join(image_directory, filename)) as img:
                for entry in bounding_boxes:
                    if entry['image_name'] == filename:
                        start_time = time.time()
                        for idx, classification in enumerate(entry['classifications']):
                            x_center, y_center = classification['nucleus_x'], classification['nucleus_y']
                            bethesda_system = adjust_class_name(classification['bethesda_system'])
                            x, y = int(x_center - 45), int(y_center - 45)
                            cropped_img = img.crop((x, y, x + 90, y + 90))
                            # Resize the image to 224x224
                            resized_img = cropped_img.resize((224, 224))
                            cropped_img.save(os.path.join(output_directory, f"{filename[:-4]}_{bethesda_system}_{idx}.png"))
                        total_cropping_time += time.time() - start_time

                        # Predict using the model
                        start_time = time.time()
                        resized_img_array = np.array(resized_img)[np.newaxis, ...]  # Adjust dimensions if needed
                        predictions = model.predict(resized_img_array)
                        total_prediction_time += time.time() - start_time
    return total_prediction_time

# Load bounding boxes from JSON
with open('classificationDownload/classifications.json', 'r') as file:
    bounding_boxes = json.load(file)

# Process images and measure time
prediction_time = process_images(image_directory, bounding_boxes)
print(f"Total Prediction Time: {prediction_time:.2f} seconds")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━

## Tiempo de Predicción de la Clasificación por Tipo de Lesión Especifica

In [ ]:
# Define directories and load model
image_directory = "base"
output_directory = "imgs/crops_90x90"
model_path = "modelosFinales/xception_clasificacion5clases.keras"  # Adjust with actual path to your pre-trained model

# Ensure output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Load the model without compiling
model = tf.keras.models.load_model(model_path, compile=False)

# Recompile the model manually with a known optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Function to adjust class name
def adjust_class_name(bethesda_system):
    return 'Negative' if bethesda_system == 'Negative for intraepithelial lesion' else bethesda_system

# Function to process images
def process_images(image_directory, bounding_boxes):
    total_cropping_time = 0
    total_prediction_time = 0
    for filename in os.listdir(image_directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            with Image.open(os.path.join(image_directory, filename)) as img:
                for entry in bounding_boxes:
                    if entry['image_name'] == filename:
                        start_time = time.time()
                        for idx, classification in enumerate(entry['classifications']):
                            x_center, y_center = classification['nucleus_x'], classification['nucleus_y']
                            bethesda_system = adjust_class_name(classification['bethesda_system'])
                            x, y = int(x_center - 45), int(y_center - 45)
                            cropped_img = img.crop((x, y, x + 90, y + 90))
                            # Resize the image to 224x224
                            resized_img = cropped_img.resize((224, 224))
                            cropped_img.save(os.path.join(output_directory, f"{filename[:-4]}_{bethesda_system}_{idx}.png"))
                        total_cropping_time += time.time() - start_time

                        # Predict using the model
                        start_time = time.time()
                        resized_img_array = np.array(resized_img)[np.newaxis, ...]  # Adjust dimensions if needed
                        predictions = model.predict(resized_img_array)
                        total_prediction_time += time.time() - start_time
    return total_cropping_time, total_prediction_time

# Load bounding boxes from JSON
with open('classificationDownload/classifications.json', 'r') as file:
    bounding_boxes = json.load(file)

# Process images and measure time
cropping_time, prediction_time = process_images(image_directory, bounding_boxes)
print(f"Total Prediction Time: {prediction_time:.2f} seconds")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━

## Tiempo de Predicción de la Clasificación de 6 Clases

In [ ]:
# Define directories and load model
image_directory = "base"
output_directory = "imgs/crops_90x90"
model_path = "modelosFinales/xception_clasificacion6clases.keras"  # Adjust with actual path to your pre-trained model

# Ensure output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Load the model without compiling
model = tf.keras.models.load_model(model_path, compile=False)

# Recompile the model manually with a known optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Function to adjust class name
def adjust_class_name(bethesda_system):
    return 'Negative' if bethesda_system == 'Negative for intraepithelial lesion' else bethesda_system

# Function to process images
def process_images(image_directory, bounding_boxes):
    total_cropping_time = 0
    total_prediction_time = 0
    for filename in os.listdir(image_directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            with Image.open(os.path.join(image_directory, filename)) as img:
                for entry in bounding_boxes:
                    if entry['image_name'] == filename:
                        start_time = time.time()
                        for idx, classification in enumerate(entry['classifications']):
                            x_center, y_center = classification['nucleus_x'], classification['nucleus_y']
                            bethesda_system = adjust_class_name(classification['bethesda_system'])
                            x, y = int(x_center - 45), int(y_center - 45)
                            cropped_img = img.crop((x, y, x + 90, y + 90))
                            # Resize the image to 224x224
                            resized_img = cropped_img.resize((224, 224))
                            cropped_img.save(os.path.join(output_directory, f"{filename[:-4]}_{bethesda_system}_{idx}.png"))
                        total_cropping_time += time.time() - start_time

                        # Predict using the model
                        start_time = time.time()
                        resized_img_array = np.array(resized_img)[np.newaxis, ...]  # Adjust dimensions if needed
                        predictions = model.predict(resized_img_array)
                        total_prediction_time += time.time() - start_time
    return total_cropping_time, total_prediction_time

# Load bounding boxes from JSON
with open('classificationDownload/classifications.json', 'r') as file:
    bounding_boxes = json.load(file)

# Process images and measure time
cropping_time, prediction_time = process_images(image_directory, bounding_boxes)
print(f"Total Prediction Time: {prediction_time:.2f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━

## Tiempo de Predicción de la Clasificación del Pipeline

In [ ]:
# Load the model
model_path = "modelosFinales/xception_clasificacion2clases.keras"
binary_model = tf.keras.models.load_model(model_path, compile=False)
binary_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Directory for test data
test_dir = 'crops/test'

# Setup the data generator for the test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Make predictions using the binary model
predictions = (binary_model.predict(test_generator) > 0.8).astype(int)

# Calculate the number of 'lesion' predictions
lesion_predictions_count = np.sum(predictions)
total_predictions = len(predictions)

# Calculate the proportion of 'lesion' predictions
proportion_with_lesion = lesion_predictions_count / total_predictions

print(f"Total images predicted as having a lesion: {lesion_predictions_count}")
print(f"Proportion with lesion: {proportion_with_lesion:.4f} or {proportion_with_lesion * 100:.2f}%")

Found 1732 images belonging to 2 classes.
55/55 ━━━━━━━━━━━━━━━━━━━━ 54s 973ms/step
Total images predicted as having a lesion: 734
Proportion with lesion: 0.4238 or 42.38%


El enfoque del pipeline consiste en utilizar primero un modelo binario que clasifica todas las células en dos categorías: con lesión y sin lesión. Posteriormente, solo el subconjunto de células clasificadas como con lesión se procesa a través del modelo de clasificación de 5 clases para especificar el tipo de lesión. La muestra utilizada no tienen una proporción de celulas cancerigenas representativa a la realidad (42.38%) por lo que se asume que el 100% de las celulas son cancerigenas y pasaran por el modelo de 5 clases. Aunque en la aplicación no vayan a ser todas las celulas cancerigenas, se determina que el tiempo maximo de predicción del pipeline por imagen es la suma de los tiempos promedios que tardara cada modelo en clasificar una imagen.

Tiempo de Predicción por imagen = Tiempo de Predicción por imagen del Binario + Tiempo de Predicción por imagen del de 5 Clases 

Tiempo de Predicción por imagen = 0.1006 + 0.0982 = 0.1988

Tiempo de Predicción Total = 0.1988 * 400 = 79.52

# Tabla comparativa 

In [51]:
# Define a list to hold all the rows of the table
table = []
per_image_table = []
# Define headers for the table
headers = ["Modelos", "Total Cropping Time (s) for 400 images", "Total Prediction Time (s) for 400 images", "Total Time (s) for 400 images"]
per_image_headers = ["Modelos", "Average Cropping Time (s) per image", "Average Prediction Time (s) per image", "Average Total Time (s) per image"]

def add_model_data(model_name, cropping_time, prediction_time):
    """
    Function to add a model's data to the table.
    """
    total_time = cropping_time + prediction_time
    table.append([model_name, cropping_time, prediction_time, total_time])
    # Calculate per image time
    per_image_time = [cropping_time / 400, prediction_time / 400, total_time / 400]
    per_image_table.append([model_name] + per_image_time)

def display_table():
    """
    Function to display the total time table.
    """
    # Print header
    print("{:<25} {:<35} {:<35} {:<35}".format(*headers))
    # Print row data
    for row in table:
        print("{:<25} {:<35.2f} {:<35.2f} {:<35.2f}".format(*row))

def display_per_image_table():
    """
    Function to display the per-image time table.
    """
    # Print header
    print("{:<25} {:<35} {:<35} {:<35}".format(*per_image_headers))
    # Print row data
    for row in per_image_table:
        print("{:<25} {:<35.4f} {:<35.4f} {:<35.4f}".format(*row))

# Example usage:
add_model_data("Clasificación 2 Clases", 31.28, 40.24)
add_model_data("Clasificación 3 Clases", 31.28, 37.95)
add_model_data("Clasificación 5 Clases", 31.28, 39.30)
add_model_data("Clasificación 6 Clases", 31.28, 41.99)
add_model_data("Clasificación de Pipeline", 31.28, 79.52)


# Display the tables
print("Table for Total Times for 400 Images:")
display_table()
print("\nTable for Average Times Per Image:")
display_per_image_table()



Table for Total Times for 400 Images:
Modelos                   Total Cropping Time (s) for 400 images Total Prediction Time (s) for 400 images Total Time (s) for 400 images      
Clasificación 2 Clases    31.28                               40.24                               71.52                              
Clasificación 3 Clases    31.28                               37.95                               69.23                              
Clasificación 5 Clases    31.28                               39.30                               70.58                              
Clasificación 6 Clases    31.28                               41.99                               73.27                              
Clasificación de Pipeline 31.28                               79.52                               110.80                             

Table for Average Times Per Image:
Modelos                   Average Cropping Time (s) per image Average Prediction Time (s) per image Average Total 